In [1]:
import pandas as pd
df = pd.read_csv( '1.csv', sep=',', encoding='utf-8' )

In [2]:
df

,name,phone,group_id,question_id,confirm,result,result_comment
0,Куринов Максим,89093869470,3,1,1,0,Согласен/ 3 раза звонила не взял трубку
1,Баж Егор,89047510599,3,1,1,0,Согласен/ тел не доступен
2,Сериков Андрей,89081499151,1,1,1,0,Согласен/ не выход
3,Егоров Андрей,89009250730,1,1,1,1,Согласен/ выход
4,Птицын Валентин,89685221822,1,1,1,1,Согласен/ выход
...,...,...,...,...,...,...,...
95,Бойка Игорь Анатольевич,89009027134,1,6,1,0,Согласен/ 3 раза звонила не взял трубку
96,Есипов Николай Владиславович,89250528281,1,6,1,1,Согласен/выход
97,Аксенов Александр,89521283717,1,6,1,1,Согласен/выход
98,Владислав Владислав,89225718763,1,6,1,1,Согласен/выход


In [48]:
def mark_dataset(df):
    df['result'] = df['result'].astype('bool')
    df['phone'] = df['phone'].astype('string')
    
    # type of "lambda x" is pandas.DataFrame obj type
    df['audio_script_json']=df['phone'].apply(lambda x: pd.read_json('audio/' + x + '.mp3.json'))
    
    # type of j['full_transcript'] and j['detaled'] is pandas.Series
    # type of j['full_transcript'][0] and j['detaled'][0] is Dict
    
    df['sym_len'] = df['audio_script_json'].apply( lambda j: len(j['full_transcript'][0]) )
    df['words_len'] = df['audio_script_json'].apply( lambda j: len(j['full_transcript'][0].split()) )
    df['dialog_df'] = df['audio_script_json'].apply( lambda j: pd.json_normalize( j['detaled'] ))
    
    # df['operator_negs_df'] = df['dialog_df'].loc[ df['dialog_df']["gender"] == "female", ["negative_sentiment"] ].squeeze()
    # df['master_negs_df'] = df['dialog_df'].loc[ df['dialog_df']["gender"] == "male", ["negative_sentiment"] ].squeeze()
    
    # df['master_negs_df'] = df['dialog_df'].apply( lambda d: d.loc[ d["gender"] == "male", ["negative_sentiment"] ].squeeze() )
    # df['operator_negs_df'] = df['dialog_df'].apply( lambda d: d.loc[ d["gender"] == "female", ["negative_sentiment"] ].squeeze() )
    
    #df['neg_sent_master_max'] = df['master_negs_df'].max()
    #df['neg_sent_master_avg'] = df['master_negs_df'].mean()
    #df['neg_sent_operator_max'] = df['operator_negs_df'].max()
    #df['neg_sent_operator_avg'] = df['operator_negs_df'].mean()
    
    
    df['neg_sent_master_max'] = df['dialog_df'].apply( lambda d: d.loc[ d["gender"] == "male", ["negative_sentiment"] ].squeeze().max() )
    df['neg_sent_master_avg'] = df['dialog_df'].apply( lambda d: d.loc[ d["gender"] == "male", ["negative_sentiment"] ].squeeze().mean() )
    
    df['neg_sent_operator_max'] = df['dialog_df'].apply( lambda d: d.loc[ d["gender"] == "female", ["negative_sentiment"] ].squeeze().max() )
    df['neg_sent_operator_avg'] = df['dialog_df'].apply( lambda d: d.loc[ d["gender"] == "female", ["negative_sentiment"] ].squeeze().mean() )
    
#     for x in ["neg_sent_master_max", "neg_sent_master_avg", "neg_sent_operator_max", "neg_sent_operator_avg"]:
#         df[x] = 100*df[x]
    
    # pd.json_normalize(df.iloc[:1]['audio_script_json'][0]['detaled'])
    # df['sym_len'] = df['audio_script_json'].apply( lambda j: print( j.info() ))
    
    df = df[ fields ]
    return df

df = mark_dataset(df)

df.info()
df

NameError: name 'fields' is not defined

In [43]:
# some debut output
x = 0

In [44]:
df.iloc[x]["dialog_df"].head(4)

KeyError: 'dialog_df'

In [31]:
test_series = df.iloc[x]["dialog_df"].loc[ df.iloc[x]["dialog_df"]["gender"] == "female", ["negative_sentiment"] ].squeeze()
test_series

1     0.471525
3     0.000000
4     0.000000
6     0.228629
7     0.185469
8     0.000000
9     0.199823
11    0.000000
14    0.474421
15    0.000000
16    0.000000
20    0.472630
22    0.299953
26    0.000000
Name: negative_sentiment, dtype: float64

In [32]:
test_series.describe()

count    14.000000
mean      0.166604
std       0.195874
min       0.000000
25%       0.000000
50%       0.092735
75%       0.282122
max       0.474421
Name: negative_sentiment, dtype: float64

In [33]:
# type(df.iloc[:1]['audio_script_json'])     # pandas.core.series.Series
# len(df.iloc[:1]['audio_script_json'][0].full_transcript[0])
# df.iloc[:1]
#df['audio_script_json'].info()
# len(df.iloc[:1]['audio_script_json'][0]['full_transcript'][0])
# df.iloc[:1]['audio_script_json'][0]['full_transcript']
# df.iloc[:1]['audio_script_json'][0]
# df.iloc[:1]['audio_script_json'][0]['detaled'][0]['phrase']
# df.iloc[:1]['audio_script_json'][0]['detaled']
# df.iloc[:1]['audio_script_json'][0]['detaled'].query('negative_sentiment > 0')  # pandas.Series object type

df_dialog = pd.json_normalize(df.iloc[:1]['audio_script_json'][0]['detaled'])
df_dialog
# выделяем 4 параметра: neg_sent_master_max, neg_sent_master_avg, neg_sent_operator_max, neg_sent_operator_avg

,phrase,negative_sentiment,gender
0,здравствуйте,0.000000,male
1,это сервис майгур поступила заявка на ремонт к...,0.471525,female
2,а кто это,0.000000,male
3,сервис майгур,0.000000,female
4,а,0.000000,female
5,кабельные трассы а что то можно будет сделать,0.000000,male
6,кабельная трасса подземная работа открытые то ...,0.228629,female
7,так как трасса уже раскопана спецтехникой рабо...,0.185469,female
8,всего нужно до пять человек,0.000000,female
9,вы выйдете один на данный заказ или у вас буде...,0.199823,female


In [34]:
# df_dialog.loc[df_dialog['gender'] == 'female', ['phrase', 'negative_sentiment', 'gender']].min()
df_dialog.loc[df_dialog['gender'] == 'female', ['negative_sentiment']].squeeze().max()
df_dialog.loc[df_dialog['gender'] == 'female', ['negative_sentiment']].squeeze().mean()

0.16660358863217475

In [35]:
df

,name,phone,group_id,question_id,confirm,result,result_comment,audio_script_json,sym_len,words_len,dialog_df,neg_sent_master_max,neg_sent_master_avg,neg_sent_operator_max,neg_sent_operator_avg
0,Куринов Максим,89093869470,3,1,1,False,Согласен/ 3 раза звонила не взял трубку,full_tra...,958,152,...,0.000000,0.000000,47.442111,16.660359
1,Баж Егор,89047510599,3,1,1,False,Согласен/ тел не доступен,full_tra...,871,139,...,10.952574,2.226332,13.785648,5.154991
2,Сериков Андрей,89081499151,1,1,1,False,Согласен/ не выход,full_tra...,596,91,...,41.615990,11.326808,44.775635,13.819638
3,Егоров Андрей,89009250730,1,1,1,True,Согласен/ выход,full_tra...,850,132,...,47.828418,9.727217,32.029909,16.077423
4,Птицын Валентин,89685221822,1,1,1,True,Согласен/ выход,full_tra...,850,132,...,47.828418,9.727217,32.029909,16.077423
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Бойка Игорь Анатольевич,89009027134,1,6,1,False,Согласен/ 3 раза звонила не взял трубку,full_tra...,1282,205,...,35.312858,6.170920,33.229643,7.812440
96,Есипов Николай Владиславович,89250528281,1,6,1,True,Согласен/выход,full_tra...,1539,267,...,66.674900,7.459662,52.156729,8.728209
97,Аксенов Александр,89521283717,1,6,1,True,Согласен/выход,full_tra...,1803,310,...,60.320270,13.720624,46.204317,18.695628
98,Владислав Владислав,89225718763,1,6,1,True,Согласен/выход,full_tra...,820,128,...,22.416300,5.071473,38.610268,12.870089


In [49]:
fields_to_leave=[
    'group_id',
    'question_id',
    'sym_len',
    'words_len',
    'neg_sent_master_max',
    'neg_sent_master_avg',
    'neg_sent_operator_max',
    'neg_sent_operator_avg',
    'result']
df = df [ fields_to_leave ]
df

,group_id,question_id,sym_len,words_len,neg_sent_master_max,neg_sent_master_avg,neg_sent_operator_max,neg_sent_operator_avg,result
0,3,1,958,152,0.000000,0.000000,0.474421,0.166604,False
1,3,1,871,139,0.109526,0.022263,0.137856,0.051550,False
2,1,1,596,91,0.416160,0.113268,0.447756,0.138196,False
3,1,1,850,132,0.478284,0.097272,0.320299,0.160774,True
4,1,1,850,132,0.478284,0.097272,0.320299,0.160774,True
...,...,...,...,...,...,...,...,...,...
95,1,6,1282,205,0.353129,0.061709,0.332296,0.078124,False
96,1,6,1539,267,0.666749,0.074597,0.521567,0.087282,True
97,1,6,1803,310,0.603203,0.137206,0.462043,0.186956,True
98,1,6,820,128,0.224163,0.050715,0.386103,0.128701,True


In [38]:
from sklearn.linear_model import LogisticRegression
import numpy as np

param_to_predict='result'
y = df[param_to_predict]
X = df.loc[:, df.columns != param_to_predict]
X

,group_id,question_id,sym_len,words_len,neg_sent_master_max,neg_sent_master_avg,neg_sent_operator_max,neg_sent_operator_avg
0,3,1,958,152,0.000000,0.000000,47.442111,16.660359
1,3,1,871,139,10.952574,2.226332,13.785648,5.154991
2,1,1,596,91,41.615990,11.326808,44.775635,13.819638
3,1,1,850,132,47.828418,9.727217,32.029909,16.077423
4,1,1,850,132,47.828418,9.727217,32.029909,16.077423
...,...,...,...,...,...,...,...,...
95,1,6,1282,205,35.312858,6.170920,33.229643,7.812440
96,1,6,1539,267,66.674900,7.459662,52.156729,8.728209
97,1,6,1803,310,60.320270,13.720624,46.204317,18.695628
98,1,6,820,128,22.416300,5.071473,38.610268,12.870089


In [41]:
LR = LogisticRegression(random_state=0, solver='liblinear').fit(X, y)
print(round(LR.score(X,y), 4))
result = LR.predict( pd.DataFrame( data = { 'group_id': 3, 'question_id': 5 }, index = [0] ) )
result

0.7


ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- neg_sent_master_avg
- neg_sent_master_max
- neg_sent_operator_avg
- neg_sent_operator_max
- sym_len
- ...
